In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.utils as vutils
from torch.optim import SGD
import torch.utils.data as Data
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
##使用手写字体数据，准备训练数据集
train_data = torchvision.datasets.MNIST(
    root="D:\Rookie\pytorch\data\MNIST",  ##数据的路径
    train=True,  ##只使用训练的数据集
    ##将数据转化为torch使用的张量，取值范围为[0,1]
    transform=torchvision.transforms.ToTensor(),
    download=True
)
##定义一个数据加载器
train_loader = Data.DataLoader(
    dataset=train_data,  ##使用的数据集
    batch_size=128,  ##批处理样本大小
    shuffle=True,  ##每次迭代前打乱数据
    num_workers=2,  ##使用两个进程
)
##准备需要使用的测试数据集
test_data = torchvision.datasets.MNIST(
    root="D:\Rookie\pytorch\data\MNIST",  ##数据的路径
    train=False,
    download=False
)
##为数据添加一个通道维度，并且取值范围缩放的0~1之间
test_data_x = test_data.data.type(torch.FloatTensor) / 255.0
test_data_x = torch.unsqueeze(test_data_x, dim=1)
test_data_y = test_data.targets  ##测试集标签
print("test_data_x.shape:", test_data_x.shape)
print("test_data_y.shape:", test_data_y.shape)

In [ ]:
##搭建一个卷积神经网络
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        ##定义第一个卷积层
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,  ##输入的feature map
                out_channels=16,  ##输出的feature map
                kernel_size=3,  ##卷积核尺寸
                stride=1,  ##卷积核步长
                padding=1,  ##进行填充
            ),
            nn.ReLU(),  ##激活函数
            nn.AvgPool2d(
                kernel_size=2,  ##平均池化层尺寸
                stride=2,  ##池化步长
            ),
        )
        ##定义第二个卷积层
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  ##最大池化
        )
        ##定义全连接层
        self.fc = nn.Sequential(
            nn.Linear(
                in_features=32 * 7 * 7,  ##输入特征
                out_features=128,  ##输入特征数
            ),
            nn.ReLU(),  #激活函数
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.out = nn.Linear(64, 10)  ##进行最后的分类层

    ##定义网络的前向传播路径
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  ##展开多维的卷积图层
        x = self.fc(x)
        output = self.out(x)
        return output

In [ ]:
##输出网络结构
MyConvnet = ConvNet()
print(MyConvnet)

In [ ]:
import hiddenlayer as h1

##可视化卷积神经网络
h1_graph = h1.build_graph(MyConvnet, torch.zeros([1, 1, 28, 28]))
help(h1.graph)
# h1_graph.theme = h1_graph.theme.copy()
##将可视化的网络保存为图片
h1_graph.save("D:\Rookie\pytorch\data\MyConvent_h1.png", format="png")

In [ ]:
from torchviz import make_dot
##使用make_dot可视化网络
x=torch.randn(1,1,28,28).requires_grad_(True)
y=MyConvnet(x)
MyConvnetvis=make_dot(y,params=dict(list(MyConvnet.named_parameters())+[("x",x)]))
##将MyConvnetvis保存为图片
MyConvnetvis.format="png"##形式转化为png
##指定文件保存位置
MyConvnetvis.directory="../data/Myconvnet_vis"
MyConvnetvis.view()##会自动在当前文件夹生成文件